In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/apsrtc"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages(driver):
    for page in range(1, 6):  # There are 5 pages
        try:
            load_page(driver, URL)
            
            if page >= 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Initialize the driver outside the loop
driver = initialize_driver()

# Scrape routes and details from all pages
scrape_all_pages(driver)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('Andhra_bus_details.csv', index=False)

# Close the driver after scraping is done
if driver:
    driver.quit()


Error occurred while accessing page 1: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.91)
Stacktrace:
0   chromedriver                        0x0000000102fa4500 cxxbridge1$str$ptr + 1917112
1   chromedriver                        0x0000000102f9c890 cxxbridge1$str$ptr + 1885256
2   chromedriver                        0x0000000102bac538 cxxbridge1$string$len + 89424
3   chromedriver                        0x0000000102b87980 core::str::slice_error_fail::hbaf5d05fe3921cd2 + 3776
4   chromedriver                        0x0000000102c17234 cxxbridge1$string$len + 526924
5   chromedriver                        0x0000000102c2a5c4 cxxbridge1$string$len + 605660
6   chromedriver                        0x0000000102be5374 cxxbridge1$string$len + 322444
7   chromedriver                        0x0000000102be5fc4 cxxbridge1$string$len + 325596
8   chromedriver                        0x0000000102f6bd2c cxxbridge1$str$ptr +

In [5]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,TGSRTC - 14462,eGARUDA,10:35,04h 25m,15:00,3.5,INR 533,12 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,TGSRTC - 2961,RAJDHANI (A.C. Semi Sleeper),10:35,05h 55m,16:30,2.5,INR 533,25 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,TGSRTC - 16262,eGARUDA,10:40,06h 20m,17:00,2.6,INR 533,7 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,TGSRTC - 41431,Super Luxury (Non AC Seater 2+2 Push Back),10:40,05h 30m,16:10,4.1,INR 440,24 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,TGSRTC - 40277,Super Luxury (Non AC Seater 2+2 Push Back),11:00,06h 30m,17:30,2.1,INR 440,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
1676,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,SEVEN HILLS TRAVELS,A/C Sleeper (2+1),21:30,06h 35m,04:05,4.7,INR 1699,16 Seats available
1677,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,BSR Tours And Travels,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:30,05h 45m,00:15,4.3,INR 2080,21 Seats available
1678,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,AR & BCVR Travels,Non A/C Seater / Sleeper (2+1),21:30,07h 00m,04:30,4.2,INR 1000,32 Seats available
1679,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...,Moksha Travels,A/C Sleeper (2+1),17:00,07h 10m,00:10,0,INR 1299,18 Seats available


In [11]:
print("bus_routes_link")

bus_routes_link
